# 1. Install Package

In [3]:
%conda info --envs

# conda environments:
#
base                     /opt/conda


Note: you may need to restart the kernel to use updated packages.


In [4]:
%conda install contextlib

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - contextlib

Current channels:

  - https://repo.anaconda.com/pkgs/main/linux-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/linux-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.



Note: you may need to restart the kernel to use updated packages.


In [6]:
%conda install subprocess

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - subprocess

Current channels:

  - https://repo.anaconda.com/pkgs/main/linux-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/linux-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.



Note: you may need to restart the kernel to use updated packages.


# 2. Amazon Polly

In [10]:
import boto3

from botocore.exceptions import BotoCoreError, ClientError
from contextlib import closing
import os
import sys
import subprocess
from tempfile import gettempdir

bucket_name = 'aigc-bj-team1'
    
def en_text_to_speech(input_text, polly_client, s3_client):
    try:
        # Request speech synthesis
        response = polly_client.synthesize_speech(Text=input_text, OutputFormat="mp3",
                                            VoiceId="Joanna")
    except (BotoCoreError, ClientError) as error:
        # The service returned an error, exit gracefully
        print(error)
        sys.exit(-1)
    
    # Access the audio stream from the response
    if "AudioStream" in response:
        # Note: Closing the stream is important because the service throttles on the
        # number of parallel connections. Here we are using contextlib.closing to
        # ensure the close method of the stream object will be called automatically
        # at the end of the with statement's scope.
        with closing(response["AudioStream"]) as stream:
            output = os.path.join(gettempdir(), "speech.mp3")

            try:
                # Open a file for writing the output as a binary stream
                with open(output, "wb") as file:
                    file.write(stream.read())
                    print("Written to %s" % output)
                    s3_client.upload_file(output, bucket_name, "speech-en.mp3")
            except IOError as error:
                # Could not write to file, exit gracefully
                print(error)
                sys.exit(-1)
    else:
        # The response didn't contain audio data, exit gracefully
        print("Could not stream audio")
        sys.exit(-1)

def cn_text_to_speech(input_text, polly_client, s3_client):
    try:
        # Request speech synthesis
        response = polly_client.synthesize_speech(Text=input_text, OutputFormat="mp3",
                                            VoiceId="Zhiyu")
    except (BotoCoreError, ClientError) as error:
        # The service returned an error, exit gracefully
        print(error)
        sys.exit(-1)
    
    # Access the audio stream from the response
    if "AudioStream" in response:
        # Note: Closing the stream is important because the service throttles on the
        # number of parallel connections. Here we are using contextlib.closing to
        # ensure the close method of the stream object will be called automatically
        # at the end of the with statement's scope.
        with closing(response["AudioStream"]) as stream:
            output = os.path.join(gettempdir(), "speech.mp3")

            try:
                # Open a file for writing the output as a binary stream
                with open(output, "wb") as file:
                    file.write(stream.read())
                    print("Written to %s" % output)
                    s3_client.upload_file(output, bucket_name, "speech-cn.mp3")
            except IOError as error:
                # Could not write to file, exit gracefully
                print(error)
                sys.exit(-1)
    else:
        # The response didn't contain audio data, exit gracefully
        print("Could not stream audio")
        sys.exit(-1)
        
def main():
    polly_client = boto3.client('polly')
    s3_client = boto3.client("s3")
    en_input_text  = 'Hello world!'
    en_text_to_speech(en_input_text, polly_client, s3_client)
    cn_input_text = '你好'
    cn_text_to_speech(cn_input_text, polly_client, s3_client)

if __name__ == '__main__':
    main()

Written to /tmp/speech.mp3
Written to /tmp/speech.mp3


In [11]:
# Copy to local
!aws s3 cp s3://aigc-bj-team1/speech-en.mp3 ./speech-en.mp3
!aws s3 cp s3://aigc-bj-team1/speech-cn.mp3 ./speech-cn.mp3

download: s3://aigc-bj-team1/speech-en.mp3 to ./speech-en.mp3  
download: s3://aigc-bj-team1/speech-cn.mp3 to ./speech-cn.mp3  


In [12]:
from IPython.display import Audio

# Replace 'path_to_your_mp3.mp3' with the path to your mp3 file
audio_file = 'speech-en.mp3'

Audio(audio_file)

In [12]:
from IPython.display import Audio

# Replace 'path_to_your_mp3.mp3' with the path to your mp3 file
audio_file = 'speech-cn.mp3'

Audio(audio_file)